# Table of Content
1. [Set up environment](#Set-up-environment)
2. [Load RAG dataset from huggingface](#Load-RAG-dataset-from-huggingface)
3. [Build retrieval index](#Build-retrieval-index)
4. [Split dataset into train and validation parts](#Split-dataset-into-train-and-validation-parts)
5. [Construct baseline RAG Sequencemodel from pretrained blocks](#Construct-baseline-RAG-Sequencemodel-from-pretrained-blocks)
6. [Mesaure RAG quality of baseline model](#Mesaure-RAG-quality-of-baseline-model)
7. [Use Cherry Pick to get intuition](#Use-Cherry-Pick-to-get-intuition)
8. [Fine-tune RAG Sequence Model](#Fine-tune-RAG-Sequence-Model)
9. [Measure RAG quality after fine-tuning](#Measure-RAG-quality-after-fine-tuning)

## References:
- original paper https://arxiv.org/pdf/2005.11401
- RAGAS metric library https://docs.ragas.io/en/latest/getstarted/index.html

# Set up environment

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

import nest_asyncio
nest_asyncio.apply()

import dotenv
assert dotenv.load_dotenv()

assert os.getenv('MISTRALAI_API_KEY') or os.getenv('OPENAI_API_KEY'), "you need specify MISTRALAI_API_KEY or OPENAI_API_KEY in tutorials/.evn"


# Load RAG dataset from huggingface

In [2]:
# model for answer generations, will be trained via Peft
GENERATOR_MODEL = 'NousResearch/Hermes-2-Theta-Llama-3-8B'
# model for question embeddings calculation, will be trained
ENCODER_MODEL = 'abacaj/llama-161M-100B'

In [3]:
import os
import random
import torch
import numpy as np

from turbo_alignment.cherry_picks.rag import RagCherryPickCallback
from turbo_alignment.dataset.chat.chat import ChatDataset
from turbo_alignment.settings.datasets.base import (
    DatasetSourceSettings,
    DatasetStrategy,
)
from turbo_alignment.settings.datasets.chat import ChatDatasetSettings
from turbo_alignment.settings.pipelines.train.rag import RAGTrainExperimentSettings

from transformers import (
    AutoTokenizer,
    PreTrainedTokenizerBase,
    AutoModelForCausalLM)

from datasets import load_dataset, Value

/root/.cache/pypoetry/virtualenvs/turbo-alignment-QH-OqmFL-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_settings" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(
INFO:datasets:PyTorch version 2.1.2 available.


In [4]:
tokenizer=AutoTokenizer.from_pretrained(GENERATOR_MODEL)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# examine chat template of GENERATOR_MODEL to fullfill `ChatTemplate` accordingly
tokenizer.chat_template['default']

"{{bos_token}}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"

In [6]:
ds_qap = load_dataset("rag-datasets/rag-mini-bioasq", "question-answer-passages")
ds_corpus = load_dataset("rag-datasets/rag-mini-bioasq", "text-corpus")

# Build retrieval index

In [7]:
question_encoder = AutoModelForCausalLM.from_pretrained(ENCODER_MODEL)
question_encoder.cuda()
question_encoder.eval()
question_tokenizer = AutoTokenizer.from_pretrained(ENCODER_MODEL)

In [8]:
from turbo_alignment.modeling.rag.utils import get_question_embeddings
def embed(sentence: str)->np.ndarray:
    '''
    calculates sentence embedding via ENCODER_MODEL
    '''
    with torch.no_grad():
        input_ids = question_tokenizer.encode(sentence, return_tensors='pt')
        input_ids=input_ids.cuda()
        attention_mask = torch.ones_like(input_ids)
        encoder_output = question_encoder(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        embedding = get_question_embeddings(encoder_output, attention_mask)
        embedding = embedding.reshape(-1).cpu().numpy()

    return embedding

In [9]:
len(ds_corpus['passages'])

40221

In [10]:
# calc embeddings of passages and build fiass index for passages retrieval

num_passages=30000 # use None for whole dataset or change to number < len(ds_corpus['passages']) to speed up processing
if num_passages is not None:
    passages_with_embeddings = ds_corpus['passages'].select(range(num_passages))
else:
    passages_with_embeddings = ds_corpus['passages']

passages_with_embeddings = passages_with_embeddings.map(lambda example: {'embeddings': embed(example['passage'])})

passages_with_embeddings.add_faiss_index(column='embeddings')

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


  0%|          | 0/30 [00:00<?, ?it/s]

Dataset({
    features: ['passage', 'id', 'embeddings'],
    num_rows: 30000
})

In [22]:
# save faiss index to disk
passages_with_embeddings.save_faiss_index('embeddings', 'my_index.faiss')
passages_with_embeddings.drop_index('embeddings')

MissingIndex: Index with index_name 'embeddings' not initialized yet. Please make sure that you call `add_faiss_index` or `add_elasticsearch_index` first.

In [12]:
# rename columns to be suitable for RAGCherryPickCallback

passages_with_embeddings = passages_with_embeddings.rename_column("id", "title").rename_column('passage', 'text')

features = passages_with_embeddings.features.copy()
features['title'] = Value('string')
passages_with_embeddings = passages_with_embeddings.cast(features)


Casting the dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [13]:
# save passages datasets to load it as a part of RAGSequenceForGeneration
passages_with_embeddings.save_to_disk('passages')

Saving the dataset (0/1 shards):   0%|          | 0/30000 [00:00<?, ? examples/s]

# Split dataset into train and validation parts

In [14]:
# records are dicts compatible with ChatDatasetRecord
# they are suitable for 'ChatDataset' creation

records = []
for example in ds_qap['test']:
    messages = [{'role': 'user', 'content': example['question']}, {'role': 'bot', 'content': example['answer']}]
    record = {
        "messages": messages,
        "meta": {'relevant_passage_ids': example['relevant_passage_ids']},
        "id": example['id'],
    }

    records.append(record)

In [15]:
num_val_records=30 # change to -1 to use all validational records, may be slow
random.seed(0)
random.shuffle(records)
train_num_samples = int(0.8 * len(records))
train_records = records[:train_num_samples]
val_records = records[train_num_samples:]
val_records = val_records[:num_val_records]

In [16]:
# prompt template corresponds to tokenizer.chat_template we examined early
PROMPT_TEMPLATE={
            "role_tag_mapping": {"bot": "<bot>", "user": "<user>", "system": "<system>"},
            "prefix_template": "<|im_start|>{role}\n",
            "suffix_template": "<|im_end|>",
        }

from turbo_alignment.dataset.registry import DatasetRegistry

def records_to_dataset(records: list, strategy: str, tokenizer: PreTrainedTokenizerBase):
    '''
    build ChatDataset from records list
    '''
    dataset_cls = DatasetRegistry.by_name('chat').by_name(strategy)

    source = DatasetSourceSettings(name="rag-mini-bioasq", records_data=records, num_samples=len(records))

    chat_dataset_settings_dict = {
        "prompt_template": PROMPT_TEMPLATE,
        "dataset_type": "chat",
        "max_tokens_count": None,
        "only_answer_loss": True,
    }
    chat_dataset_settings = ChatDatasetSettings(**chat_dataset_settings_dict)

    dataset = dataset_cls(
        source=source, settings=chat_dataset_settings, tokenizer=tokenizer, read=True
    )

    return dataset


# Construct baseline RAG Sequencemodel from pretrained blocks

In [17]:
from turbo_alignment.cherry_picks.rag import RagCherryPickCallback
from turbo_alignment.dataset.chat.chat import ChatDataset
from turbo_alignment.settings.datasets.base import (
    DatasetSourceSettings,
    DatasetStrategy,
)
from turbo_alignment.settings.datasets.chat import ChatDatasetSettings

from transformers import (
    PreTrainedModel,
    BitsAndBytesConfig,
    PreTrainedTokenizerBase
)

from peft.peft_model import PeftModelForCausalLM
from peft import get_peft_config


In [18]:
model_settings_json = {
    "generator_settings": {
        "model_path": GENERATOR_MODEL,
        "model_type": "causal",
        "transformers_settings": {},
        "embeddings_initialization_strategy": {
            "<|begin_of_text|>": "<s>",
            "<|end_of_text|>": "</s>",
            "<bot>": "bot",
            "<user>": "user",
            "<system>": "system",
        },
        "peft_settings": {
            "r": 4,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "target_modules": ["q_proj", "v_proj", "k_proj"],
            "task_type": "CAUSAL_LM",
            "modules_to_save": [],
            "name": "LORA"
        },
    },
    "question_encoder_settings": {
        "model_path": ENCODER_MODEL,
        "model_type": "encoder",
        "transformers_settings": {},
        "embeddings_initialization_strategy": {},
    },
    "index_settings": {"index_path": "my_index.faiss", "passages_path": "passages"},
    "retrieval_settings": {"n_docs": 5, "max_doc_length": 256, "query_encoder_max_length": 128},
}

from turbo_alignment.modeling.rag.rag_model import RagSequenceForGeneration
from turbo_alignment.modeling.rag.rag_tokenizer import RagTokenizer
from turbo_alignment.settings.rag_model import RAGPreTrainedModelSettings

model_settings = RAGPreTrainedModelSettings.model_validate(model_settings_json)

quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
generator = AutoModelForCausalLM.from_pretrained(GENERATOR_MODEL, quantization_config=quantization_config)

rag_tokenizer = RagTokenizer(model_settings, tokenizer_path=None)

rag_model = RagSequenceForGeneration(model_settings, generator, question_encoder, rag_tokenizer)


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


rag_index.py:104 [INFO] 2024-07-17T13:40:01+00:00 Loading passages from passages
retriever_model.py:56 [INFO] 2024-07-17T13:40:01+00:00 initializing retrieval
rag_index.py:115 [INFO] 2024-07-17T13:40:01+00:00 Loading index from my_index.faiss


# Mesaure RAG quality of baseline model

In [19]:
# build CherryPick object

from turbo_alignment.cherry_picks.rag import RagCherryPickCallback
from turbo_alignment.metrics.metric import Metric
from turbo_alignment.metrics.registry import RagasMetricsSettings
from turbo_alignment.settings.cherry_pick import ChatCherryPickSettings
from turbo_alignment.dataset.registry import DatasetRegistry


cherry_pick_settings_dict = {
        "generator_transformers_settings": {
            "num_beams": 1,
            "max_new_tokens": 256,
            "do_sample": False,
        },
        "custom_generation_settings": {"generation_eos_token": "<|im_end|>", "skip_special_tokens": True},
        "dataset_settings": {
            "sources": [
                {
                    "name": "support",
                    "records_path": "tests/fixtures/datasets/chat/train_chat_rag.jsonl",
                    "num_samples": 1,
                }
            ],
            "prompt_template": {
                "role_tag_mapping": {"bot": "<bot>", "user": "<user>", "system": "<system>"},
                "prefix_template": "<|im_start|>{role}",
                "suffix_template": "<|im_end|>",
            },
            "dataset_type": "chat",
            "max_tokens_count": None,
            "random_cut": False,
            "only_answer_loss": True,
        },
        "metric_settings": [],
    }

cherry_pick_settings = ChatCherryPickSettings.model_validate(cherry_pick_settings_dict)


ragas_metrics = Metric.by_name("ragas_metrics")(
        settings=RagasMetricsSettings(mistralai_api_key=os.getenv('MISTRALAI_API_KEY'),
    openai_api_key=os.getenv('OPENAI_API_KEY'), need_average=[False])
    )

inference_chat_dataset = records_to_dataset(val_records, 'inference', tokenizer)

rag_cherry_pick = RagCherryPickCallback(
    cherry_pick_settings, datasets=[inference_chat_dataset], metrics=[ragas_metrics]
)

    

base.py:60 [INFO] 2024-07-17T13:40:02+00:00 Sampling 30 from dataset rag-mini-bioasq
chat.py:240 [INFO] 2024-07-17T13:40:02+00:00 Tokenizing dataset rag-mini-bioasq
chat.py:260 [INFO] 2024-07-17T13:40:02+00:00 Postprocessing tokenized data in rag-mini-bioasq
base.py:48 [INFO] 2024-07-17T13:40:02+00:00 Sampled 30 records with offset None


/root/.cache/pypoetry/virtualenvs/turbo-alignment-QH-OqmFL-py3.10/lib/python3.10/site-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [ ]:
output = rag_cherry_pick.on_evaluate(None, None, None, tokenizer=tokenizer, model=rag_model)
print(output)

In [ ]:
# RAGS metrics before fine-tuning
output[-1][-1]

 # Use Cherry Pick to get intuition

In [ ]:
# show cherry picks
sample_idx=2

print('Question')
print(output[0][0].element_wise_scores[0].values[sample_idx])
print('---')
print('Answer')
print(output[0][1].element_wise_scores[0].values[sample_idx])
print('---')
print('Documents')
print(output[0][2].element_wise_scores[0].values[sample_idx])
print('---')
print('Scores')
print(output[0][3].element_wise_scores[0].values[sample_idx])

# Fine-tune RAG Sequence Model

In [15]:
# make generator trainable via PEFT

peft_config=model_settings.generator_settings.peft_settings.dict()
peft_config["peft_type"]= peft_config['name']
del peft_config['name']

peft_config=get_peft_config(peft_config)
generator = PeftModelForCausalLM(generator, peft_config)

rag_model = RagSequenceForGeneration(model_settings, generator, question_encoder, rag_tokenizer)

rag_index.py:104 [INFO] 2024-07-17T13:14:06+00:00 Loading passages from passages
retriever_model.py:56 [INFO] 2024-07-17T13:14:06+00:00 initializing retrieval
rag_index.py:115 [INFO] 2024-07-17T13:14:06+00:00 Loading index from my_index.faiss


In [16]:
from transformers import (
    AdamW,
    PreTrainedModel,
    BitsAndBytesConfig,
    PreTrainedTokenizerBase,
    TrainingArguments)

kwargs = {
        'output_dir':'train_rag_output',
        'evaluation_strategy': 'epoch',
        'save_strategy': 'epoch',
        'per_device_train_batch_size': 2,
        'per_device_eval_batch_size': 1,
        'gradient_accumulation_steps': 1,
        'eval_steps': 150,
        'save_steps': 150,
        'logging_steps': 1,
        'learning_rate': 0.0004,
        'num_train_epochs': 1,
        'max_steps': 200,
        'lr_scheduler_type': 'linear',
        'lr_scheduler_kwargs': {},
        'warmup_steps': 0,
        'warmup_ratio': 0.1,
        'fp16': True,
        'bf16': False,
        'tf32': False,
        'torch_compile': False,
        'optim': 'adamw_torch',
        'adam_beta1': 0.9,
        'adam_beta2': 0.98,
        'adam_epsilon': 1e-06,
        'weight_decay': 0.01,
        'max_grad_norm': 0.11,
        'deepspeed': None,
        'save_total_limit': 1,
        'save_only_model': False,
        'no_cuda': False,
        'prediction_loss_only': False,
        'load_best_model_at_end': True,
        'logging_first_step': True,
        'fsdp_config': None,
        'fsdp': '',
        'dataloader_num_workers': 8,
        'dataloader_prefetch_factor': None,
        'dataloader_persistent_workers': False,
        'dataloader_pin_memory': True,
        'gradient_checkpointing': False,
        'gradient_checkpointing_kwargs': {},
        'neftune_noise_alpha': None,
        'report_to': [],
    }
training_args = TrainingArguments(**kwargs)

training_args._n_gpu=1

/root/.cache/pypoetry/virtualenvs/turbo-alignment-QH-OqmFL-py3.10/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# create train dataset and val dataset

train_dataset = records_to_dataset(train_records, 'train', tokenizer)
val_dataset = records_to_dataset(val_records, 'inference', tokenizer)

base.py:60 [INFO] 2024-07-17T13:14:22+00:00 Sampling 3775 from dataset rag-mini-bioasq
chat.py:240 [INFO] 2024-07-17T13:14:22+00:00 Tokenizing dataset rag-mini-bioasq
chat.py:260 [INFO] 2024-07-17T13:14:22+00:00 Postprocessing tokenized data in rag-mini-bioasq
base.py:48 [INFO] 2024-07-17T13:14:23+00:00 Sampled 3775 records with offset None
base.py:60 [INFO] 2024-07-17T13:14:23+00:00 Sampling 10 from dataset rag-mini-bioasq
chat.py:240 [INFO] 2024-07-17T13:14:23+00:00 Tokenizing dataset rag-mini-bioasq
chat.py:260 [INFO] 2024-07-17T13:14:23+00:00 Postprocessing tokenized data in rag-mini-bioasq
base.py:48 [INFO] 2024-07-17T13:14:23+00:00 Sampled 10 records with offset None


In [22]:
# create trainer
from transformers.data.data_collator import DataCollatorForTokenClassification, DataCollatorForSeq2Seq

data_collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=8)

from turbo_alignment.trainers.multigpu import MultiGPUCherryPicksTrainer

os.environ['TOKENIZERS_PARALLELISM']='false'

trainer = MultiGPUCherryPicksTrainer(
    model=rag_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
generator_parameters = rag_model.rag.generator.parameters()
question_encoder_parameters = rag_model.rag.question_encoder.parameters()
optimizer = AdamW(
    [
        #we want to train retriever rather then generator, since lr for generator 100 lower
        {'params': generator_parameters, 'lr': training_args.learning_rate/100},
        {
            'params': question_encoder_parameters,
            'lr': training_args.learning_rate,
        },
    ]
)
trainer.optimizer = optimizer

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/root/.cache/pypoetry/virtualenvs/turbo-alignment-QH-OqmFL-py3.10/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|                                                                                                         …

{'loss': 1040.5778, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 832.3809, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 499.2353, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 772.701, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 606.6097, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 1022.1019, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 405.2975, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 253.0243, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.0}
{'loss': 421.7507, 'grad_norm': 384.970458984375, 'learning_rate': 2e-07, 'epoch': 0.0}
{'loss': 362.9537, 'grad_norm': nan, 'learning_rate': 2e-07, 'epoch': 0.01}
{'loss': 924.6088, 'grad_norm': nan, 'learning_rate': 2e-07, 'epoch': 0.01}
{'loss': 538.185, 'grad_norm': 566.7750854492188, 'learning_rate': 4e-07, 'epoch': 0.01}
{'loss': 532.3833, 'grad_norm': 610.6382446289062, 'learning_rate': 6e-07, 'epoch': 0.

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.83 GiB. GPU 0 has a total capacty of 79.35 GiB of which 1.79 GiB is free. Process 1758960 has 77.55 GiB memory in use. Of the allocated memory 69.15 GiB is allocated by PyTorch, and 7.90 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Measure RAG quality after fine-tuning

In [19]:
output = rag_cherry_pick.on_evaluate(None, None, None, tokenizer=tokenizer, model=trainer.model)
print(output)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


start parsing ```json
{"entities": ["gastro esophageal reflux disease", "LPR (Laryngopharyngeal reflux)", "intraoral burning sensations"]}
``` with prompt prompt_str='Given a text, extract unique entities without repetition. Ensure you consider different forms or mentions of the same entity as a single entity.\n\nThe output should be a well-formatted JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output JSON schema:\n```\n{"type": "object", "properties": {"entities": {"title": "Entities", "type": "array", "items": {"type": "string"}}}, "required": ["entities"]}\n```\n\nDo not return any preamble or explanations, return only a pure JSON st

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


start parsing ```json
{"entities": ["Familial craniosynostosis", "Pro250Arg mutation", "fibroblast growth factor receptor 3 gene", "skin disorders", "psoriatic lesions", "XRT", "tumor site", "Gy", "colchicine", "cell cycle", "H2B gene family", "Top Down Mass Spectrometry", "Electron Capture Dissociation", "MS/MS", "H2B isoforms", "unmodified H2B.Q", "H2B.A", "H2B.K/T", "H2B.J", "H2B.E", "H2B.B", "H2B.F", "monoacetylated H2B.A", "sodium butyrate treatment", "Muenke syndrome", "coronal craniosynostosis", "sensorineural deafness", "craniofacial abnormalities", "digital abnormalities", "upper airway obstruction"]}
``` with prompt prompt_str='Given a text, extract unique entities without repetition. Ensure you consider different forms or mentions of the same entity as a single entity.\n\nThe output should be a well-formatted JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type

...

start parsing ```
[
  {
    "sentence_index": 0,
    "simpler_statements": [
      "Sorafenib activates AMP-activated protein kinase (AMPK).",
      "Sorafenib is a tyrosine kinase inhibitor."
    ]
  },
  {
    "sentence_index": 1,
    "simpler_statements": [
      "AMPK is a cellular energy sensor.",
      "AMPK plays a crucial role in regulating cellular energy homeostasis, metabolism, and autophagy."
    ]
  },
  {
    "sentence_index": 2,
    "simpler_statements": [
      "Sorafenib is used to treat liver cancer.",
      "Sorafenib activates AMPK through the inhibition of ATR and the activation of the AMPKα subunit."
    ]
  },
  {
    "sentence_index": 3,
    "simpler_statements": [
      "The activation of AMPK by sorafenib may contribute to its anticancer effects.",
      "AMPK can inhibit cell proliferation, promote autophagy, and induce apoptosis in cancer cells."
    ]
  },
  {
    "sentence_index": 4,
    "simpler_statements": [
      "The activation of AMPK by sorafenib ma

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


start parsing [
{"statement": "Sorafenib activates AMP-activated protein kinase (AMPK).", "reason": "There is no information in the context about Sorafenib or AMP-activated protein kinase (AMPK).", "verdict": 0},
{"statement": "Sorafenib is a tyrosine kinase inhibitor.", "reason": "There is no information in the context about Sorafenib or its properties.", "verdict": 0},
{"statement": "AMPK is a cellular energy sensor.", "reason": "There is no information in the context about AMPK or its functions.", "verdict": 0},
{"statement": "AMPK plays a crucial role in regulating cellular energy homeostasis, metabolism, and autophagy.", "reason": "There is no information in the context about AMPK or its functions.", "verdict": 0},
{"statement": "Sorafenib is used to treat liver cancer.", "reason": "There is no information in the context about Sorafenib or its uses in cancer treatment.", "verdict": 0},
{"statement": "Sorafenib activates AMPK through the inhibition of ATR and the activation of the 

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


start parsing ```[
  {
    "statement": "Ribociclib is an inhibitor of cyclin D-cyclin-dependent kinase 4/6 (CDK 4/6).",
    "attributed": 0,
    "reason": "The context does not mention anything about ribociclib being an inhibitor of cyclin D-cyclin-dependent kinase 4/6 (CDK 4/6)."
  },
  {
    "statement": "Ribociclib is used for breast cancer treatment.",
    "attributed": 0,
    "reason": "The context does not mention anything about ribociclib being used for breast cancer treatment."
  }
]
``` with prompt prompt_str='Given a context, and an answer, analyze each sentence in the answer and classify if the sentence can be attributed to the given context or not. Use only "Yes" (1) or "No" (0) as a binary classification. Output json with reason.\n\nThe output should be a well-formatted JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string

In [20]:
# show cherry picks after fine tuning
sample_idx=2

print('Question')
print(output[0][0].element_wise_scores[0].values[sample_idx])
print('---')
print('Answer')
print(output[0][1].element_wise_scores[0].values[sample_idx])
print('---')
print('Documents')
print(output[0][2].element_wise_scores[0].values[sample_idx])
print('---')
print('Scores')
print(output[0][3].element_wise_scores[0].values[sample_idx])

Question
[user: Which growth factors are known to be involved in the induction of EMT?]
---
Answer
1: Several growth factors have been shown to be involved in the induction of epithelial-mesenchymal transition (EMT), including:

1. Transforming growth factor-beta (TGF-β): TGF-β is a potent inducer of EMT and plays a crucial role in the development of cancer metastasis.
2. Epidermal growth factor (EGF): EGF stimulates EMT in various cell types, including cancer cells, and promotes tumor invasion and metastasis.
3. Platelet-derived growth factor (PDGF): PDGF is known to induce EMT in various cell types, including cancer cells, and promotes tumor progression and metastasis.
4. Fibroblast growth factor (FGF): FGF signaling has been shown to induce EMT in various cell types, including cancer cells, and promote tumor invasion and metastasis.
5. Hepatocyte growth factor (HGF): HGF is a potent inducer of EMT and plays a crucial role in the development of cancer metastasis.
6. Vascular endothel

In [21]:
output[-1][-1]

{'faithfulness': 0.0071, 'answer_relevancy': 0.9429, 'answer_similarity': 0.9176, 'context_precision': 0.1756, 'context_recall': 0.0750, 'context_entity_recall': 0.0000}